In [ ]:
import nltk
import sys
import os 
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import nltk.data
import string
from pyemd import emd
import numpy as np
from gensim import models
from gensim.models import word2vec
import numpy.linalg as la
from gensim.models.word2vec import Word2Vec
import scipy.spatial as spt
from scipy.spatial import distance
from pyemd import emd
from sklearn.metrics import euclidean_distances
from numpy import dot, zeros, dtype, array, sqrt,double,array,sqrt, sum as np_sum
import time
from collections import Counter
from scipy.spatial import distance
from scipy.interpolate import Rbf
from scipy.spatial.distance import pdist, squareform


In [ ]:
def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            r.append(os.path.join(root, name))
    numberOfFiles = len(r)
    return r
path = os.path.join(os.path.expanduser('.'),'summaries','docs')
fnames = list_files(path)
numberOfFiles = len(os.listdir(path))

In [ ]:
#FirstLineAllPara....processedFirstLineAllPara,,,,, 
#processedAllList,,,,,rawAllList....
#LastSentEachFile...processedLastSentEachFile

paraFile = []
processedAllList= []
rawAllList =[]
firstSentence = []
lastSentence = []
FirstLineAllPara = []
LastPara =[]
LastSentEachFile = []
for name in (fnames):
    with open(name, 'r') as f:
        data = f.read().lower()
        paragraphs = data.split("\n\n")
        paragraphs[:] = (value for value in paragraphs if value != '\t')
        LastPara.append(paragraphs[-1])
    paraFile.extend(paragraphs)
for i in range(len(LastPara)):
    LasteachPara =[]
    for j in (LastPara[i].strip().split('. ')):
        LasteachPara.append(j)
    LastSentEachFile.append(LasteachPara[-1])
processedLastSentEachFile = []
for i in range(len(LastSentEachFile)):
    line = LastSentEachFile[i]
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    Str2 = ('\n-----\n'.join(sent_detector.tokenize(line.strip())))
    Str2 = re.sub(' +', ' ', Str2)
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(Str2)
    Str1 = " ".join(filter(lambda word: word not in stop_words, Str2.split()))
    Str1 = re.sub(r'[^\w\s]','',Str1) 
    if(Str1 != ''):
        processedLastSentEachFile.append(Str1)

for para in range(len(paraFile)):
    eachPara = paraFile[para]
    for firstLine in (eachPara.strip().split('. ')):
        FirstLineAllPara.append(firstLine)
        break     
processedFirstLineAllPara = []
for i in range(len(FirstLineAllPara)):
    line = FirstLineAllPara[i]
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    Str2 = ('\n-----\n'.join(sent_detector.tokenize(line.strip())))
    Str2 = re.sub(' +', ' ', Str2)
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(Str2)
    Str1 = " ".join(filter(lambda word: word not in stop_words, Str2.split()))
    Str1 = re.sub(r'[^\w\s]','',Str1) 
    if(Str1 != ''):
        processedFirstLineAllPara.append(Str1)
processedList= []
rawList = []   
RawList = []
sents = None
text1 = None
text1 = paraFile
text =[]
for elem in text1:
    text.extend(elem.strip().split('. '))  

sent = [el.replace('\n', '') for el in text]
if(sents ==None):
    sents = sent
else:
    sents = sents+sent
rawList = sents
for i in range(len(rawList)):
    l = rawList[i]
    if(l != ''):
        RawList.append(l)
rawAllList = rawAllList+ RawList
processedAllList =[]
for i in range(len(rawAllList)):
    line = rawAllList[i]
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    Str2 = ('\n-----\n'.join(sent_detector.tokenize(line.strip())))
    Str2 = re.sub(' +', ' ', Str2)
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(Str2)
    Str1 = " ".join(filter(lambda word: word not in stop_words, Str2.split()))
    Str1 = re.sub(r'[^\w\s]','',Str1) 
    if(Str1 != ''):
        processedList.append(Str1)

    processedAllList.append(Str1)        


In [ ]:
word_vec = None
array = None
global words_1
global words_2
global words_1_set
global words_2_set
removed_words = ['sdkls']
W = np.memmap("data/embed.dat", dtype=np.double, mode="r", shape=(3000000, 300))
with open("data/embed.vocab") as f:
    vocab_list = map(str.strip, f.readlines())
vocab_dict = {w: k for k, w in enumerate(vocab_list)}

In [ ]:
def FileOpen(line1,line2):  
    global woduplicates
    global words_1
    woduplicates = None
    words_1 = line1.split()
    words_1 = [token for token in words_1 if token in vocab_dict.keys()]
    global words_2
    words_2 = line2.split()
    words_2 = [token for token in words_2 if token in vocab_dict.keys()] 
    woduplicates = set(words_1 + words_2)
    return woduplicates
def Dictionaty2Files(content, woduplicates):
    doc_len = len(content)
    d = []
    for i, t in enumerate(woduplicates):
        d.append(content.count(t) / float(doc_len))
    return d 
def distance_matrix():
    vocab_len = len(woduplicates)
    distance_matrix = zeros((vocab_len, vocab_len), dtype=float)
    for i, t1 in enumerate(woduplicates):
        for j, t2 in enumerate(woduplicates):
            distance_matrix[i][j] = sqrt(np_sum((W[[vocab_dict[t1]]] - W[[vocab_dict[t2]]])**2))
    return distance_matrix


In [ ]:
lenFirst = len(processedFirstLineAllPara)
print(len(processedList))
lenAll = len(processedList)
distanceM =np.zeros((lenAll,lenAll))
for i in range(lenAll):
    WMD_DICT = dict()
    Line = processedList[i]
    for j in range(lenAll):
        Line1 = processedList[j]
        woduplicates = FileOpen(Line1,Line)
        if(len(words_2)>0 and len(words_1)>0):
            d1 = np.array(Dictionaty2Files(words_1, woduplicates), dtype=double)
            d2 = np.array(Dictionaty2Files(words_2, woduplicates), dtype=double)
            distance = distance_matrix()
            distanceWMD = emd(d1,d2,distance)
            WMD_DICT.update({(rawAllList[i],distanceWMD )})
            distanceM[i][j] = distanceWMD

            

In [ ]:
SentFirstLine = []
for row in range(lenFirst):
    minRow = max(i for i in distanceM[row] if i > 0)
    indesOfValuemin = np.where(distanceM[row] == minRow)
    iandj = ([x[0] for x in indesOfValuemin])
    i = iandj[0]
    SentFirstLine.append(rawAllList[i])
print(set(SentFirstLine))